In [1]:

import requests
import pandas as pd
import pymongo
#from config import api_key
from pprint import pprint
from sqlalchemy import create_engine


In [2]:
url = "https://developer.nps.gov/api/v1/places?"
api_key = "zOHVmRCcC47t0WzbO6PqCg3UUOLlhcvcAO0YaLC0"

In [3]:
#test_pull = requests.get(url+"&api_key="+api_key).json()
#test_pull

In [4]:
#pprint(test_pull)

In [5]:
#print(test_pull['data'][0]['title'])

In [6]:
#states = ['CA','AK']

In [7]:
params = {
    #"stateCode": state,
    "limit" : 950,
    "api_key": api_key,
    }

response = requests.get(url,params=params).json()

In [8]:
#pprint(response)

In [4]:
places = response['data']
#pprint(places)

In [10]:
park_detail = []
for park in places:
    park_dict = {}
    try:
        name = park['relatedParks'][0]['fullName']
        lat = park['latitude']
        lon = park['longitude']
        parkUrl = park['url']
        parkid = park['id']
        parkcode = park['relatedParks'][0]['parkCode']
        designation = park['relatedParks'][0]['designation']
        imageURL = park['images'][0]['url']
        state = park['relatedParks'][0]['states']
        coordinates = park['latLong']

        park_dict['park'] = name
        park_dict['latitude'] = lat
        park_dict['longitude'] = lon
        park_dict['url'] = parkUrl
        park_dict['park_id'] = parkid
        park_dict['park_code'] = parkcode
        park_dict['park_type'] = designation
        park_dict['image'] = imageURL
        park_dict['state'] = state
        park_dict['coord'] = coordinates


        park_detail.append(park_dict)

        print('--------')
        print(name, lat, lon, parkUrl, parkid, parkcode, designation, imageURL, state, coordinates)

    except:
        print('----')
        print("failed")


--------
Pictured Rocks National Lakeshore   https://www.nps.gov/places/miners-castle.htm A17C6BB6-FE40-42EC-ACB8-EF5A6BE3DA7E piro National Lakeshore https://www.nps.gov/common/uploads/cropped_image/1F732017-C17A-EA0B-EF8EF884819DB59D.jpg MI 
--------
Mammoth Cave National Park 37.19327420021557 -86.10334360000695 https://www.nps.gov/places/green-river-bluffs-overlook.htm D3531106-B29D-442D-94F3-B01DF70EAC40 maca National Park https://www.nps.gov/common/uploads/cropped_image/61E692D2-FAFD-213F-013BE4361EEE5109.jpg KY 37.19327420021557,-86.10334360000695
--------
Little Rock Central High School National Historic Site   https://www.nps.gov/places/elizabeth-eckford-bus-bench.htm CB66E326-BC14-4E81-A6A3-EF7A2D874FAB chsc National Historic Site https://www.nps.gov/common/uploads/cropped_image/BF2B20A8-E13D-2477-E4360B1EECD1CCCB.jpeg AR 
--------
Little Rock Central High School National Historic Site   https://www.nps.gov/places/south-junction-of-front-curvilinear-sidewalks-at-central-high-

In [11]:
park_df = pd.DataFrame(park_detail)
park_df.count()
park_df[['latitude', 'longitude']] = park_df[['latitude', 'longitude']].apply(pd.to_numeric)

In [39]:

park_df.dtypes

park          object
latitude     float64
longitude    float64
url           object
park_id       object
park_code     object
park_type     object
image         object
state         object
coord         object
dtype: object

In [12]:
park_df.head(5)

,park,latitude,longitude,url,park_id,park_code,park_type,image,state,coord
0,Pictured Rocks National Lakeshore,NaN,NaN,https://www.nps.gov/places/miners-castle.htm,A17C6BB6-FE40-42EC-ACB8-EF5A6BE3DA7E,piro,National Lakeshore,https://www.nps.gov/common/uploads/cropped_ima...,MI,
1,Mammoth Cave National Park,37.193274,-86.103344,https://www.nps.gov/places/green-river-bluffs-...,D3531106-B29D-442D-94F3-B01DF70EAC40,maca,National Park,https://www.nps.gov/common/uploads/cropped_ima...,KY,"37.19327420021557,-86.10334360000695"
2,Little Rock Central High School National Histo...,NaN,NaN,https://www.nps.gov/places/elizabeth-eckford-b...,CB66E326-BC14-4E81-A6A3-EF7A2D874FAB,chsc,National Historic Site,https://www.nps.gov/common/uploads/cropped_ima...,AR,
3,Little Rock Central High School National Histo...,NaN,NaN,https://www.nps.gov/places/south-junction-of-f...,5760ED27-E5DA-47C5-A76F-1F83FB769267,chsc,National Historic Site,https://www.nps.gov/common/uploads/cropped_ima...,AR,
4,Little Rock Central High School National Histo...,NaN,NaN,https://www.nps.gov/places/north-curvilinear-f...,6CA3DA56-4062-4848-B49F-D9C66F377D13,chsc,National Historic Site,https://www.nps.gov/common/uploads/cropped_ima...,AR,


In [13]:
rds_connection_string = "postgres:admin@localhost:5432/group_2_project_3"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [14]:
engine.table_names()

/var/folders/4s/j3_l93ts54qgnyg57c9s590m0000gn/T/ipykernel_14873/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['parks', 'webcams']

In [15]:
park_df.to_sql(name='parks', con=engine, if_exists='append', index=False)

In [16]:
pd.read_sql_query('select * from parks', con=engine).head()

,park,latitude,longitude,url,park_id,park_code,park_type,image,state,coord
0,Pictured Rocks National Lakeshore,NaN,NaN,https://www.nps.gov/places/miners-castle.htm,A17C6BB6-FE40-42EC-ACB8-EF5A6BE3DA7E,piro,National Lakeshore,https://www.nps.gov/common/uploads/cropped_ima...,MI,
1,Mammoth Cave National Park,37.0,-86.0,https://www.nps.gov/places/green-river-bluffs-...,D3531106-B29D-442D-94F3-B01DF70EAC40,maca,National Park,https://www.nps.gov/common/uploads/cropped_ima...,KY,"37.19327420021557,-86.10334360000695"
2,Little Rock Central High School National Histo...,NaN,NaN,https://www.nps.gov/places/elizabeth-eckford-b...,CB66E326-BC14-4E81-A6A3-EF7A2D874FAB,chsc,National Historic Site,https://www.nps.gov/common/uploads/cropped_ima...,AR,
3,Little Rock Central High School National Histo...,NaN,NaN,https://www.nps.gov/places/south-junction-of-f...,5760ED27-E5DA-47C5-A76F-1F83FB769267,chsc,National Historic Site,https://www.nps.gov/common/uploads/cropped_ima...,AR,
4,Little Rock Central High School National Histo...,NaN,NaN,https://www.nps.gov/places/north-curvilinear-f...,6CA3DA56-4062-4848-B49F-D9C66F377D13,chsc,National Historic Site,https://www.nps.gov/common/uploads/cropped_ima...,AR,
